## 검증세트(validation set)
* 테스트 세트를 사용하지 않으면 모델이 과대적합인지 과소적합인지 판단하기 어려움
* 테스트를 사용하지 않고 이를 측정하는 간단한 방법은 훈련 세트를 또 나누는 방법 이것을 검증세트라고 부름
* 이전에서 훈련세트 80%, 테스트세트 20%했고 이번에는 훈련 세트 60%, 테스트세트 20%, 검증세트 20%로 진행
* 데이터 세트가 많은 경우 일부 몇 %만 떼어도 상관없음

In [2]:
import pandas as pd
wine = pd.read_csv('http://bit.ly/wine_csv_data')

In [3]:
## 타겟 값 따로 저장
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [4]:
## 훈련 세트와 테스트 세트 데이터 나누기
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size = 0.2, random_state = 42)
print(train_input.shape, test_input.shape)

(5197, 3) (1300, 3)


In [5]:
## 훈련세트의 20% 검증 데이터셋으로 사용하기
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size = 0.2, random_state= 42)

In [6]:
## 훈련 세트 5197에서 4157로 줄고, 검증 세트 1040개 생성
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [7]:
## 훈련세트 과적합
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state = 42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


## 교차검증
* 검정세트를 만드느라 훈련세트가 줄어듬
* 보통 많은 데이터를 훈련에 사용할수록 좋은 모델이 만들어짐
* 검증 세트를 너무 조금 떼어 놓으면 검증 점수가 들쭉날쭉하고 불안정할 것임
* 이럴 때 교차검증을 이용하면 안정적인 검증 점수를 얻고 훈련에 더 많은 데이터를 사용할 수 있다.
### 3-Fold 교차검증

| 훈련세트 | 훈련세트 | 검증세트 | 
| --- | --- | --- |
| 훈련세트 | 검증세트 | 훈련세트 | 
| 검증세트 | 훈련세트 | 훈련세트 |

* 각각 3번의 모델평가를 통해 검증 점수 평균을 내는 방식이다

In [10]:
## 사이킷런의 교차 검증 함수 사용
## 이 함수
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00592399, 0.00496101, 0.00462484, 0.00615001, 0.00479221]), 'score_time': array([0.00058293, 0.00050807, 0.00050926, 0.00066185, 0.00051308]), 'test_score': array([0.87019231, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [12]:
## 각각 모델의 훈련하는 시간과 검증하는 시간, 5개의 검증 폴더의 점수를 나타냄(test점수와 혼동x)
pd.DataFrame(scores, 
             columns=["fit_time", "score_time", "test_score"])

,fit_time,score_time,test_score
0,0.005924,0.000583,0.870192
1,0.004961,0.000508,0.846154
2,0.004625,0.000509,0.876805
3,0.006150,0.000662,0.848893
4,0.004792,0.000513,0.835419


In [13]:
## 검증 데이터의 평균 점수
import numpy as np
print(np.mean(scores['test_score']))

0.8554925223957948


#### 주의할 점
* cross_validate()는 훈련 세트를 섞어 폴드를 나누지 않음
* 이전에는 train_test_split()을 사용했기 때문에 섞을 필요가 없었음
* 만약 교차 검증을 할 때 훈련 세트를 섞으려면 분할기(splitter)를 지정해야 함
* 사이킷런의 분할기는 교차 검증에서 폴드를 어떻게 나눌지 결정해 줌
* 회귀 모델의 경우 KFold 분할기를 사용
* 분류 모델일 경우 stratifiedKFold를 사용


In [14]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv = StratifiedKFold())

In [16]:
## 10-Fold 교차 검증
splitter = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)
scores = cross_validate(dt, train_input, train_target, cv = splitter)
print(np.mean(scores['test_score']))

0.8581873425226026


####  하이퍼파라미터 튜닝
* 머신러닝 모델이 학습하는 파라미터를 모델의 파라미터라고 부름
* 모델이 학습할 수 없어서 사용자가 지정해야만 하는 파라미터를 하이퍼파라미터라고 부름
##### 최적의 하이퍼파라미터를 어떻게 찾나?
> 결정 트리 모델에서 최적의 max_depth 값을 찾고 min_samples_split을 바꿔가면서 최적의 값을 찾는 경우 불행하게도 max_depth의 최적값은 min_samples_split 매개변수의 값이 바뀌면 함께 달라진다. 즉 이 두 매개변수를 동시에 바꿔가며 최적의 값을 찾아야 함 게다가 매개변수가 많아지면 문제는 더 복잡해짐
for 문을 사용해서 찾을 수 있지만 사이킷런에서 제공하는 **그리드 서치**를 사용하면 됨
>
##### GridSearchCV
* 하이퍼파라미터 탐색과 교차 검증을 한 번에 수행함
* cross_validate() 함수를 호출할 필요가 없음

In [18]:
## GridSearchCv 클래스로 min_impurity_decrease 매개변수의 최적값 찾기
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease' : [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [21]:
## 기본값인 5로 폴드를 만들어서 진행하고 n_jobs는 -1로 모든 코어를 사용함
gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), params, n_jobs = -1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [24]:
## 가장 성능이 좋은 파라미터로 성능 확인
dt = gs.best_estimator_
print(dt.score(train_input, train_target))
print(gs.best_params_)
print(gs.cv_results_['mean_test_score'])

0.9615162593804117
{'min_impurity_decrease': 0.0001}
[0.86800067 0.86453617 0.86492226 0.86780891 0.86761605]


In [26]:
## argmax로 직접 최적의 파라미터 확인 (동일함)
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


##### 과정 정리
1. 먼저 탐색할 매개변수를 지정
2. 훈련 세트에서 그리드 서치를 수행하여 최상의 평균 검증 점수가 나오는 매개변수 조합을 찾음 (조합은 그리드 서치 객체에 저장)
3. 그리드 서치는 최상의 매개변수에서 전체 훈련 세트를 사용해 최종 모델을 훈련함(이 모델도 그리드 서치 객체에 저장됨)

In [32]:
## 복잡한 매개변수 조합으로 탐색 (9x15x10 = 1350개, 기본 5폴드 총 모델의 수는 6750개)
params = {'min_impurity_decrease': np.arange(0.00001, 0.0001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)}

In [33]:
## 다시 학습
gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), params, n_jobs = -1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([1.e-05]),
                         'min_samples_split': range(2, 100, 10)})

In [35]:
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

{'max_depth': 12, 'min_impurity_decrease': 1e-05, 'min_samples_split': 72}
0.8637695269119716


#### 랜덤서치
* 매개변수의 값이 수치일 때 값의 범위나 간격을 미리 정하기 어려울 수 있다.
* 또 너무 많은 매개변수 조건이 있어 그리드 서치 수행 시간이 오래 걸릴 수 있다 이럴 때 랜던서치를 사용
* 매개변수 값의 목록을 전달하는 것이 아니라 매개변수를 샘플링할 수 있는 확률 분포 객체를 전달

In [37]:
## 싸이파이에서 2개의 확률 분포 클래스 임포트
## 싸이파이는 파이썬의 핵심 과학 라이브러리중 하나로 적분, 보간, 선형 대수, 확률 등을 포함한 수치 계산 전용 라이브러리
from scipy.stats import uniform, randint ## 균등 분포에서 샘플링
rgen = randint(0, 10)
rgen.rvs(10)



array([8, 6, 4, 4, 8, 6, 3, 8, 7, 0])

In [38]:
np.unique(rgen.rvs(1000), return_counts = True) ## 많은 숫자를 뽑으면 균등하게 뽑히는 것을 알 수 있음

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([100, 113,  91, 100, 100,  98,  91,  92, 107, 108]))

In [39]:
ugen = uniform(0,1) ## 난수 발생기라고 생각하면 편함
ugen.rvs(10)

array([0.84750361, 0.60946798, 0.14548806, 0.22273582, 0.41604605,
       0.23607642, 0.46802385, 0.7473723 , 0.76729988, 0.94344625])

In [40]:
params = {'min_impurity_decrease' : uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25)}

In [42]:
from sklearn.model_selection import RandomizedSearchCV 
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state = 42), params,
                        n_iter = 10, n_jobs = -1, random_state = 42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x1210d5760>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x120c97cb0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x1210d70b0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x28463ff20>},
                   random_state=42)

In [43]:
print(gs.best_params_)

{'max_depth': 42, 'min_impurity_decrease': 0.00015808361216819946, 'min_samples_leaf': 24, 'min_samples_split': 22}


In [44]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683852817057822


In [45]:
## 테스트 세트 점수는 검증 세트에 대한 점수보다 조금 작은 것이 일반적임
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.8569230769230769


## 정리
### 개념
##### 검증세트
* 하이퍼파리미터 튜닝을 위해 모델을 평가할 때, 테스트 세트를 사용하지 않기 위해 훈련 세트에서 다시 떼어 낸 데이터 세트임
##### 교차검증
* 훈련 세트를 여러 폴드로 나눈 다음 한 폴드가 검증 세트의 역할을 하고 나머지 폴드에서는 모델을 훈련함 교차 검증은 이런 식으로 모든 폴드에 대해 검증 점수를 얻어 평균하는 방법
##### 그리드 서치
* 하이퍼 파라미터 탐색을 자동화해 주는 도구임
* 탐색할 매개변수를 나열하면 교차 검증을 수행하여 가장 좋은 검증 점수의 매개변수 조합을 선택함
* 마지막으로 이 매개변수 조합으로 최종 모델을 훈련함
##### 랜덤 서치
* 연속된 매개변수 값을 탐색할 때 유용
* 탐색할 값을 나열하는 것이 아니고 탐색 값을 샘플링할 수 있는 확률 분포 객체를 전달함
* 지정된 횟수만큼 샘플링하여 교차 검증을 수행하기 때문에 시스템 자원이 허락하는 만큼 탐색량을 조절할 수 있음

### 핵심 패키지와 함수
##### cross_validate()
* 교차 검증을 수행하는 함수
* scoring : 매개변수에 검증에 사용할 평가 지표를 지정(기본적으로 분류 모델은 정확도를 의미하는 'accuracy', 회귀 모델은 결정계수를 의미하는 'r2')
* cv : 매개변수에 교차 검증 폴드 수나 스플리터 객체를 지정, 기본값은 5 (회귀일 때는 KFold 클래스 사용, 분류일 때는 stratifiedKFold클래스를 사용)
* n_jobs : 교차 검증을 수행할 때 사용할 CPU 코어 수를 지정 기본값은 1, 모든 코어 사용은 -1
* return_train_score : True로 지정시 훈련 세트의 점수도 반환 기본값은 False

##### GridSearchCV
* 교차 검증으로 하이퍼파라미터 탐색을 수행, 최상의 모델을 찾은 후 훈련 세트 전체를 사용해 최종 모델을 훈련함
* scoring, cv, n_jobs, return_train_score 매개변수는 cross_validate()와 동일

##### RandomizedSearchCV
* 교차 검증으로 랜덤한 하이퍼파라미터 탐색을 수행, 최상의 모델을 찾은 후 훈련 세트 전체를 사용해 최종 모델을 훈련함
* scoring, cv, n_jobs, return_train_score 매개변수는 cross_validate()와 동일


### 퀴즈
##### 1번
* 교차 검증이란 무엇일까요?
##### 2번
* 그리드 서치와 랜덤 서치의 차이점은 무엇일까요? 
##### 3번
* 랜덤한 정수를 원하는 만큼 뽑아내는 함수는 무엇일까요?  랜덤한 실수를 원하는 만큼 뽑아내는 함수는 무엇일까요? 